In [ ]:
import pandas as pd
import numpy as np
trn_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
tst_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
tst=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')
fnl_trn=fnl_trn.drop(list(fnl_trn.filter(regex='V')),1)
fnl_tst=fnl_tst.drop(list(fnl_tst.filter(regex='V')),1)
import gc
fnl=pd.concat([fnl_trn,fnl_tst],0)
fnl['V307']=pd.concat([trn,tst])['V307']
del([trn,tst,fnl_trn,fnl_tst])
gc.collect()
import re
def number(data):
  return  ' '.join(re.findall(r'\d+',data)) 
def charac(data):
  return ''.join(re.findall("[a-zA-Z]+", data))

fnl['DeviceInfo_char']=list(map(charac,fnl['DeviceInfo'].fillna('nan')))
fnl['DeviceInfo_number']=list(map(number,fnl['DeviceInfo'].fillna('nan')))


fnl['DeviceType_char']=list(map(charac,fnl['DeviceType'].fillna('nan')))
fnl['DeviceType_number']=list(map(number,fnl['DeviceType'].fillna('nan')))


fnl['os_char']=list(map(charac,fnl['id_30'].fillna('nan')))
fnl['os_number']=list(map(number,fnl['id_30'].fillna('nan')))


fnl['browse_char']=list(map(charac,fnl['id_31'].fillna('nan')))
fnl['browse_number']=list(map(number,fnl['id_31'].fillna('nan')))


fnl['P_1']=fnl['P_emaildomain'].str.split('.').str[0]
fnl['P_2']=fnl['P_emaildomain'].str.split('.').str[1]


fnl['R_1']=fnl['R_emaildomain'].str.split('.').str[0]
fnl['R_2']=fnl['R_emaildomain'].str.split('.').str[1]

In [ ]:
used=['card'+str(i) for i in range(1,7)]
used+=['C'+str(i) for i in range(1,15)]
used+=['D'+str(i) for i in range(1,16)]
fnl[used]=fnl[used].fillna(0)fnl=fnl.reset_index(drop=True)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
numerical=['V307','TransactionAmt','dist1']
numerical+=['C'+str(i) for i in range(1,15)]
numerical+=['D'+str(i) for i in range(1,16)]
from tqdm import tqdm
for col in tqdm(dumm):
        print('before '+str(fnl[col].nunique()))
        fnl.loc[~fnl[col].isin(list(fnl[col].value_counts().index[:5])),col]='other'
        print('before '+str(fnl[col].nunique()))
        if col!='card1' or col !='id_02':
            fnl_dum=pd.get_dummies(fnl[col]).astype('uint8')
            fnl_dum.columns=list(np.char.add(col,np.asarray(list(fnl_dum)).astype(str)))
            #fnl=fnl.drop([col],1)
            fnl_dum.columns+='_dum'
            fnl=pd.concat([fnl,fnl_dum],1)


#fnl['id']=le.fit_transform(fnl['id'])
from tqdm import tqdm
for col in tqdm(numerical):
    try:
        fnl[col+'_mean']=fnl.groupby(['id'])[col].transform('mean')
        fnl[col+'_std']=fnl.groupby(['id'])[col].transform('std')
    except:
        continue
used+=['ProductCD','TransactionAmt','day','month','week','V307','addr1','dist1','DeviceInfo_char','DeviceInfo_number','DeviceType_char','DeviceType_number','os_char','os_number','browse_char','browse_number','P_1','P_2']
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7
fnl[used]=fnl[used].fillna('nan')
cats=['ProductCD','DeviceInfo_char','DeviceInfo_number','DeviceType_char','DeviceType_number','os_char','os_number','browse_char','browse_number','P_1','P_2','addr1']
dumm=['card'+str(i) for i in range(1,7)]
dumm+=cats
fnl=fnl[used]
fnl.shape

In [ ]:
your_new_df=your_new_df.loc[:,~(your_new_df.columns.duplicated())]
your_new_df.to_csv('extra_features.csv')
